In [ ]:
# processdata.ipynb (nội dung dạng script để dùng trong Jupyter Notebook)

import pandas as pd
import numpy as np
import re
import spacy
import nlpaug.augmenter.word as naw
from tqdm.notebook import tqdm, trange
import nltk

# Tải dữ liệu NLTK cần thiết
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

# Tích hợp tqdm cho pandas
tqdm.pandas()

# Load Spacy
nlp = spacy.load("en_core_web_sm")

# Hàm xử lý văn bản
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    return " ".join(tokens)

# Load dữ liệu gốc
df_true = pd.read_csv("../data/true.csv")
df_fake = pd.read_csv("../data/fake.csv")

df_true['label'] = 1
df_fake['label'] = 0

df = pd.concat([df_true, df_fake], ignore_index=True)
df['full_text'] = "Title: " + df['title'] + " | Subject: " + df['subject'] + " | Text: " + df['text']

# Làm sạch văn bản
print("🔧 Đang xử lý văn bản với tiến trình hiển thị...")
df['clean_text'] = df['full_text'].progress_apply(clean_text)

# Tăng dữ liệu bằng SynonymAug
aug = naw.SynonymAug(aug_src='wordnet')
augmented = []
print("🔁 Đang thực hiện Data Augmentation...")
for i, text in enumerate(tqdm(df['clean_text'].tolist(), desc="Augmenting", unit="text")):
    try:
        new_text = aug.augment(text)
        augmented.append(new_text)
    except Exception as e:
        print(f"⚠️ Augment lỗi ở dòng {i}: {e}")
        augmented.append(text)

# Gộp lại dữ liệu tăng cường
df_aug = df.copy()
df_aug['clean_text'] = augmented

df_combined = pd.concat([df, df_aug], ignore_index=True)

# Lưu ra file CSV
print("💾 Đang lưu cleaned_data.csv...")
df_combined.to_csv("cleaned_data.csv", index=False)
print("✅ Dữ liệu đã được xử lý và lưu vào cleaned_data.csv")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


🔧 Đang xử lý văn bản với tiến trình hiển thị...


  0%|          | 0/44898 [00:00<?, ?it/s]